# Confusion Matrix 보정

사용되는 data들은 class간 유사도가 굉장히 높다. 육안으로 구분이 어려울 정도이므로 confusing 할 가능성이 높다.

두 이미지 간 관계를 KL-Divergence로 정의하고, loss function에 가중치를 더하여 CM(Confusion Matrix) 계산 과정에서의 정확도를 보정한다.

즉, 모델이 학습 데이터를 완전히 암기하기 전 단계의 예측값과 정답을 이용하여 CM을 만든 뒤 CR을 산출하여 학습에 이용한다.

입력 이미지에 대한 모델의 예측값 ŷ 와 정답값 y를 이용하여 CM을 만든다.

이후 아래 식을 이용하여 CR을 구하고 loss function에 가중치를 더해 backpropagation을 수행한다.

CM의 행과 열은 각각 정답값과 예측값을 의미하며 i와 j는 학습할 때 클래스에 부여된 번호이다.

다음은 간단한 python pseudo code이다.

In [ ]:
from sklearn.metrics import confusion_matrix
import numpy as np
from scipy.stats import truncnorm, entropy

import math
import copy

# Random sampling (Normal distribution)
def get_truncated_normal(mean=2, sd=1, low=0, upp=5):
    return truncnorm(
        (low - mean) / sd, (upp - mean) / sd, loc=mean, scale=sd)

k = 3
epochs = 30
mini_batch = 32
num_data = 2000
prev = []

for n in range(epochs):
    y_t = get_truncated_normal()
    y_p = get_truncated_normal()
    y_true = []
    y_pred = []

    for i in range(num_data):
        y_true.append(math.floor(y_t.rvs()))
        y_pred.append(math.floor(y_p.rvs()))

    CM = [[0]*5 for _ in range(5)]
    CR = [[0.0]*5 for _ in range(5)]

    if n >= 1:
        for i in range(5):
            for j in range(5):
                if i != j:
                    CR[i][j] = float(prev[i][j]) / float(prev[i][j] + prev[j][i])
                else:
                    CR[i][j] = 0

		# Especially, regarding mini_batch indexing -> (n*mini_batch + m)
		# CM = confusion_matrix(y_true, y_pred)  : easy way
    for m in range(mini_batch): 
        y = y_true[n*mini_batch + m] # true
        y_hat = y_pred[n*mini_batch + m] # predict
        CM[y][y_hat] += 1

        if n > 1:
            for i in range(mini_batch):
                if y != y_hat:
										# Calibrate the loss when calculating.
                    Loss = Loss + k * Loss * entropy([CR[y][y_hat], CR[y_hat][y]], base=2)

    prev = copy.deepcopy(CM)